
# <span style="color:#2E86C1; font-size:50px;">MODULO ONLINE ENLACE</span>

**REQUISITOS PARA EJECUTARLO**

Una vez subido el codigo a Arduino, para proseguir con los pasos:
1) Asegurarse que tienes cargado en la misma libreria donde guardas este script el archivo con el modelo cargado. Algo así como "model.pt"

2) Ahora en este archivo debemos cambiar el "SERIAL_PORT" para que sea el puerto COM donde está el Arduino 

3) No ejecutar el script completo del tiron porque los 2 ultimos bloques sirven para detener el servidor TCP y cerrar el puerto serie.

4) ANDROID: Descarga el archivo app-debug.apk e instalalo en el movil. 

In [2]:
#LIBRERIAS
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import keyboard
import time
import os

import serial
import socket
import threading

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
from torchvision import models


In [3]:
# PATH DEFINITION


In [4]:
######## SYSTEM PARAMETERS ########
#DEMO: Sirve para testear la conexion entre Python y Android sin usar Android y con datos random.
#Si queremos testear el sistema completo debe estar como DEMO=False 
DEMO      = False
# DEMO = True --> Test the TCPServer --- AndroidDevice(IA Model) with random data generated
DEBUG     = False

######## COMMUNICATIONS ########
WIRELESS  = 'WiFi'  
#
# None: Utiliza el modelo IA instalado en el TCP Server para realizar la predicción
#    Arduino ----[USB]----TCPServer(IA Model)
#
#'WiFi': Inicia el Servidor TCP y espera la conexión del Dispositivo Android donde se ejecuta el modelo
#    Arduino ----[USB]----TCPServer------[WiFi 802.11]--------AndroidDevice(IA Model)
#
#'Bluetooth': Conecta con el dispositivo Android utilizando Bluetooth
#    Arduino ----[USB]----TCPServer------[Bluetooth]--------AndroidDevice(IA Model)

# SERIAL COM PARAMETERS
SERIAL_PORT  = 'COM7'
SERIAL_SPEED = 57600

# BLUETOOTH PARAMETERS

# WiFi PARAMETERS
TCPSERVER_HOST = '0.0.0.0'
TCPSERVER_PORT = 12345

######## IA PARAMETERS ########

# DATA PARAMETRS
W_Size = 150
INPUT_CHANNEL   = 1
CLASSES  = 4

# MODEL PARAMETERS
path = '.\\'
path_model = os.path.join(path, 'model.pt')

DROPOUT = 0

# OUTPUT
movimientos ={
    '0'  : 'REPOSO',
    '1'  : 'ABAJO',
    '2'  : 'ARRIBA',
    '3'  : 'CERRADO'
}
print('Working Path: \t', path)
print('Model Weights:\t',path_model)


Working Path: 	 .\
Model Weights:	 .\model.pt


In [5]:
# CONFIGURACION DE LA CONEXION WIRELESS CON BLUETOOTH
# NO SE UTILIZA EN ESTE PROYECTO POR ESO SE CONFIGURA A NONE
puerto_bt = None

In [6]:
# CONFIGURACION DE LA CONEXION WIRELESS CON ANDROID UTILIZANDO WiFi 802.11
# La clase TCPServer habilita un servidor TCP/IP en el puerto TCP que selecciones, en este caso: 12345

class TCPServer:
    def __init__(self, host, port):
        self.server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.port = port
        self.host = host
        self.client_socket  = None
        self.client_address = None
        hostname = socket.gethostname()
    
        # Obtener la dirección IP asociada al nombre del host
        ip_address = socket.gethostbyname(hostname)
    
        print('TCP Server Name:\t', hostname)
        print('TCP Server IP Address:\t',ip_address)
    
        
        self.clients = []
        
    def start(self):
        """Función que inicia el servidor y acepta conexiones entrantes."""
        self.server.bind((self.host, self.port))
        self.server.listen(5)
        self.lock = threading.Lock()
        self.running = threading.Event()
        self.running.set()
        print('Server Listening in:\t',self.host,':',self.port)
        
        while self.running.is_set():
            try:
                self.client_socket, client_address = self.server.accept()
                client_handler = threading.Thread(target=self.handle_client, args=(self.client_socket, client_address))
                client_handler.start()
            except socket.timeout:
                continue
            except OSError:
                break
            
    def stop(self):
        print("Stopping TCP Server at port:", self.port)
        self.running.clear()
        self.server.close()
        with self.lock:
            for client_socket, client_address in self.clients:
                client_socket.close()
                print('Socket Connection closet with client', client_address)
        self.client_socket = None
        print("TCP Server Stopped")

    def send(self, data):
        if self.client_socket is not None:
           try:
               self.client_socket.sendall(data.encode('utf-8'))
           except ConnectionResetError:
               self.client_socket = None
               print('DATA NOT SENT. Android Client Not Connected')
        return -1   
            
    def receive(self):
        if self.client_socket is not None:
            try:
                data = self.client_socket.recv(1024)
                if not data:
                    self.client_socket.close()
                    self.client_socket = None
                    with self.lock:
                        for client_socket, client_address in self.clients:
                            client_socket.close()
                            print('Socket Connection closet with client', client_address)
                                
            except ConnectionResetError:
                self.client_socket = None
                print('DATA NOT RECEIVED. Android Client Not Connected')
               
            return data.decode('utf-8')
        return None

    def is_port_in_use(self, port, host='0.0.0.0'):
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            try:
                s.bind((host, port))
                return False
            except socket.error:
                return True
    
    def handle_client(self, client_socket, client_address):
        """Función que maneja la comunicación con un cliente."""
        self.clients.append((client_socket, client_address))
        self.client_socket = client_socket
        self.client_address = client_address
        print('Client', len(self.clients), 'Connection accepted:', client_address)

######################################################


In [7]:
#MODELO PRETRAINED UTILIZADO CUANDO NO HAY CONEXION WIFI CON EL MOVIL (LOCAL)
class ConvPretrained(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ConvPretrained, self).__init__() #super:heredo de Module

        self.capa1 =  nn.Conv2d(in_channels, 3, kernel_size = (3,4), padding = (1,1))
        self.pretrained = models.resnet50(weights = models.ResNet50_Weights.DEFAULT)  #modelos distintos como resnet150, Googlenet
        for param in self.pretrained.parameters():
            param.require_grad = False
        in_features = self.pretrained.fc.in_features
        self.pretrained.fc = nn.Linear(in_features, 256)
        self.drop1 =  nn.Dropout(DROPOUT)
        self.norm1 =  nn.BatchNorm1d(256)
        self.activa1 = nn.ReLU()

        # out_channels = Número de Categorias: Número de movimientos a detectar
        self.capa2 =  nn.Linear(256, out_channels)
        self.activacion =nn.Softmax(dim = 1)


    def forward(self, x):
      #print(x.shape)
      if len(x.shape) == 1:
          x = x.reshape(1,1,x.shape[0], x.shape[1])
      elif len(x.shape) == 2:
          x = x.reshape(1,1,1, x.shape[0])
      else:
          x = x.reshape(x.shape[0],1,x.shape[1], x.shape[2])
      #print(x.shape)
      #x = x.to(torch.float64)
      x = self.capa1(x)
      x = self.pretrained(x)
      #print(x.shape)
      x = self.drop1(x)
      x = self.norm1(x) #para normalizar los datos
      x = self.activa1(x) #despues de normalizar le paso la activacion
      #Capa Dense de Salida
      x = self.capa2(x)
      #print(x.shape)
      x = self.activacion(x)
      #print(x.shape)
      return x


In [8]:
def normalize(data, type = 'std'):
    # Normalizar/estandarizar si es necesario (de acuerdo a cómo fue entrenado el modelo)
    if type == 'std':
        data = (data - data.mean())/data.std()   
    elif type == 'uniform':
        data = (data - data.min())/(data.max() - data.min())
    return data


def prediction(model, data):
    data = data.reshape((1, INPUT_CHANNEL, W_Size))
    # Realizar la predicción
    model.eval()
    with torch.no_grad():  # No necesitamos calcular los gradientes para la inferencia
      output = model(data)
    
    # Interpreta la predicción según el tipo de problema
    # Si es clasificación, podrías querer la clase con mayor probabilidad:
    probabilidades = nn.Softmax(dim=1)(output)
    prediccion = torch.argmax(probabilidades, dim=1)
    
    if DEBUG:
        print('Input Data Shape:', data.shape)
        print('Output:', output)
        print('Probabilidades:', probabilidades)
        print('Clase predicha:', prediccion)

    return str(prediccion.item())

In [9]:
def read_Data(wireless, demo):
    ###############################################################################################
    # WORKING IN DEMO MODE DATA IS RANDOM GENERATEDTO TEST THE SYSTEM
    ###############################################################################################
    if demo:
        data = torch.randn(150) 
            
    ###############################################################################################    
    # WHEN WORKING IN NORMAL MODEL, DATA ARE RECEIVED FROM SERIAL PORT (USB Connection to Arduino)
    ###############################################################################################
    else:
        data = []
        ndata = 0
    
        try:
            # Lee un conjunto de datos de tamaño dado por W_Size
            # y los guarda en una lisa
            while ndata < W_Size:
                # Lee una línea y decodifica los bytes a una cadena(string)
                dato_emg = puerto_serie.readline().decode().strip()  
                try:
                    # Convierte el dato a float y lo guarda en la lista
                    data.append(float(dato_emg))
                    ndata += 1
                except Exception:
                    continue
        
        except KeyboardInterrupt:
              print("Interrupción del usuario. Cerrando el programa.")
        
        
    data = normalize(torch.tensor(data))
    
    # When WIRELESS options are active data is sent as String CSV format
    if ((wireless == 'WiFi' and TCP_Server.client_socket)
        or wireless == 'Bluetooth'):
        sdata = ''
        for ix, d in enumerate(data):
            sdata = sdata + str(d.item())
            if ix < len(data)-1:
                sdata = sdata + ","
        sdata = sdata + "\n"
        
        return sdata
    else:
        return data
        



In [10]:
# CONSTRUYE EL MODELO
""""Se carga el modelo y carga el archivo de pesos. 
    Sirve para hacer pruebas entre ARduino y el servidor TCP sin necesidad de usar el movil. 

"""""
model =  ConvPretrained(INPUT_CHANNEL, CLASSES)

# CARGAR LOS PESOS DE MAQUINA

checkpoint = torch.load(path_model)

model.load_state_dict(checkpoint['model'])
# Configurar el Modelo en modo Evaluación para detener el entrenamiento
model.eval()

ConvPretrained(
  (capa1): Conv2d(1, 3, kernel_size=(3, 4), stride=(1, 1), padding=(1, 1))
  (pretrained): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=

In [11]:
# CONFIGURA Y HABILITA EL PUERTO SERIE PARA LA CONFIGURACION CON ARDUINO
# Emulación de un puerto serie COM a través de la conexion USB. Inicia el puerto serie
puerto_serie = None
if puerto_serie is not None and puerto_serie.is_open:
    print('Serial port' , SERIAL_PORT, 'open at', 'is already open', SERIAL_SPEED, 'bps')
elif not DEMO:
    puerto_serie = serial.Serial(SERIAL_PORT, SERIAL_SPEED)
    print('Seriap port', SERIAL_PORT, 'open at', SERIAL_SPEED, 'bps')

Seriap port COM7 open at 57600 bps


Se inicia el servidor TCP

El puerto se elige con TCPSERVER_PORT y debe coincidir con el que se utilice en el movil. 

Direccion IP y nombre es a modo informativo.

In [12]:
# CONFIGURACION DE LA CONEXION WIRELESS CON ANDROID UTILIZANDO WiFi 802.11
# Se genera un objeto de la clase TCPServer y se inicia el servidor
TCP_Server = TCPServer(TCPSERVER_HOST, TCPSERVER_PORT)
server_thread = threading.Thread(target = TCP_Server.start)
server_thread.daemon = True
server_thread.start()


TCP Server Name:	 DESKTOP-JENGSOB
TCP Server IP Address:	 192.168.1.45
Server Listening in:	 0.0.0.0 : 12345


Sirve para ver cuántos clientes se han conectado. Si me conecto con el movil debe aparecer "1". Lo que nos permite comprobar que el movil se ha conectado al servidor TCP

In [13]:
print(len(TCP_Server.clients))
print(TCP_Server.client_socket)


0
None


A continuacion, se muestra el bloque de la rutina principal. Se esperan datos del puerto serie procedentes de Arduino y cuando los tiene, los envía al móvil para que haga la prediccion y cuando tiene esta respuesta envía la prediccion de vuelta al Arduino.

La opcion de bluetooth que aparece no está implementada. 

Cuando WIRELESS = ‘WiFi’-->  Utiliza el modelo cargado en el móvil

Cuando WIRELESS = None --> Utiliza el modelo cargado en Python

Para evitar bloqueo, se ha puesto que cuando el cliente del móvil no está conectado utilice siempre el modelo cargado en Python

In [14]:
###########################################
############   MAIN ROUTINE   #############
###########################################
if TCP_Server.client_socket is None:
    client = True
else:
    client = False
    
print('Running in mode Demo:\t\t', DEMO)
print('Wireless connection active:\t',WIRELESS)
print('Wireless Client Connected:\t', client)
print('-'*50)
index=0
n_pred = []

#un maximo de 60 veces por ejemplo
while(index < 60):
    index+=1
    pred = None
    data = read_Data(wireless = WIRELESS, demo = DEMO)
    if WIRELESS == None:
        pred = prediction(model, data)    
    elif WIRELESS == 'Bluetooth':
        continue 
        #ENVIAR DATOS AL BLUETOOTH 
        puerto_bt.write(data)
        #RECIBIR RESPUESTA DE BLUETOOTH(PREDICCION)
        pred = puerto_bt.read()
    elif WIRELESS == 'WiFi' and TCP_Server.client_socket is not None:
        TCP_Server.send(data)
        pred = TCP_Server.receive().strip()
    else:
        #print('\nWireless Configured but any client or device connected--> Using Local Model')
        pred = prediction(model, data)    
    
    
    if pred is not None:
        if not pred:
            continue
        else:
            spred = movimientos[pred] 
            print( pred, end=' - ')
            n_pred.append(pred)
            
            if not DEMO :
                #pred = pred +'\n'
                puerto_serie.write(pred.encode('ascii'))
    
    if keyboard.is_pressed('esc'):
        print("Tecla ESC detectada, Deteniendo la captura...")
        break
    
    time.sleep(1) 
gesto = np.array(n_pred)
unique, count = np.unique(gesto, return_counts = True)
print("Valores de cada gesto", dict(zip(unique, count)))

    
    

Running in mode Demo:		 False
Wireless connection active:	 WiFi
Wireless Client Connected:	 True
--------------------------------------------------
1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - 1 - Valores de cada gesto {'1': 60}


In [15]:
# STOP TCP SERVER
TCP_Server.stop()


Stopping TCP Server at port: 12345
TCP Server Stopped


In [16]:
# CLOSE SERIAL PORT
if puerto_serie and puerto_serie.is_open:
    puerto_serie.close()
    print("Serial Port Closed.")

Serial Port Closed.
